In [8]:
import os
import json
import textwrap
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from utils.modeling import EmbeddingModel

client = Elasticsearch("http://127.0.0.1:9200")

# install chinese analyzer
ES_HOME = "/data/peitian/Apps/elasticsearch-8.17.0"
# !bash {ES_HOME}/bin/elasticsearch-plugin install analysis-smartcn

client.info()

with open("config/bge-m3.json") as f:
    config = json.load(f)
model = EmbeddingModel(**config)

def pretty_response(response):
    if len(response["hits"]["hits"]) == 0:
        print("Your search returned no results.")
    else:
        for hit in response["hits"]["hits"]:
            score = hit["_score"]
            content = hit["_source"]["content"]
            article = hit["_source"]["law_article"]
            section = hit["_source"]["law_section"]
            name = hit["_source"]["law_name"]
            print()
            # print(f"ID: {id}")
            print(f"Content: {textwrap.fill(content, 120)}")
            print(f"Article: {textwrap.fill(article, 120)}")
            print(f"Section: {textwrap.fill(section, 120)}")
            print(f"Name: {textwrap.fill(name, 120)}")
            print(f"Score: {score}")

In [2]:
# 创建法条index

client.indices.create(
    index="law",
    settings={
        # 'analysis': {
        #     'analyzer': {
        #         # we must set the default analyzer
        #         "default": {
        #             "type": "smartcn"
        #         }
        #     }
        # },
        # "index.mapping.ignore_malformed": True
    },
    mappings={
        "properties": {
            # field name
            "country": {
                "type": "keyword",
            },
            "law_name": {
                "type": "keyword",
            },
            "law_section": {
                "type": "keyword",
            },
            "law_article": {
                "type": "keyword",
            },
            "content": {
                "type": "text",
            },
            # "embedding": {
            #     "type": "dense_vector",
            #     "dims": 1024,
            #     # enable hnsw
            #     "index": True,
            #     # inner product only allows unit-length vector
            #     "similarity": "dot_product"  
            # }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'law'})

In [2]:
# 删除index

client.indices.delete(index="law")

ObjectApiResponse({'acknowledged': True})

In [ ]:
# 添加一带一路数据

base_dir = "/data/peitian/Data/legal/yidaiyilu/output"

def generate_laws():
    for path in os.listdir(base_dir):
        law_name = path.split(".")[0]
        with open(os.path.join(base_dir, path), encoding="utf-8") as f:
            for i, line in enumerate(f):
                break

helpers.bulk(client, generate_laws())

In [4]:
# 添加FLK数据

base_dir = "/data/peitian/Data/legal/flk/output"

def generate_laws():
    for path in os.listdir(base_dir):
        with open(os.path.join(base_dir, path), encoding="utf-8") as f:
            for i, line in enumerate(f):
                item = json.loads(line)
                item["country"] = "chn"
                item["_index"] = "law"

                yield item

helpers.bulk(client, generate_laws())

(21429, [])

In [9]:
resp = client.search(
    index="law", 
    _source={
        "excludes": "embedding",
    },
    query={
        "match": 
            {
                "content": {
                    "query": "不正当",
                    "boost": 0.2,
                }
            }
    },
    # knn={
    #     "field": "embedding",
    #     "query_vector": model.encode("这是啥")[0].tolist(),  # generate embedding for query so it can be compared to `title_vector`
    #     "k": 5,
    #     "num_candidates": 10,
    #     "boost": 0.9
    # },
    size=10,
)
pretty_response(resp)


Content: 第三十条 因正当防卫造成损害的，不承担责任。正当防卫超过必要的限度，造成不应有的损害的，正当防卫人应当承担适当的责任。
Article: 第三十条
Section: 第三章 不承担责任和减轻责任的情形
Name: 中华人民共和国侵权责任法
Score: 1.5959024

Content: 第三条 各级人民政府应当采取措施，制止不正当竞争行为，为公平竞争创造良好的环境和条件。 国务院建立反不正当竞争工作协调机制，研究决定反不正当竞争重大政策，协调处理维护市场竞争秩序的重大问题。
Article: 第三条
Section: 第一章 总则
Name: 中华人民共和国反不正当竞争法
Score: 1.510809

Content: 第十条 银行业监督管理机构工作人员，应当忠于职守，依法办事，公正廉洁，不得利用职务便利牟取不正当的利益，不得在金融机构等企业中兼任职务。
Article: 第十条
Section: 第二章 监督管理机构
Name: 中华人民共和国银行业监督管理法
Score: 1.4644626

Content: 第三十三条 在对外贸易经营活动中，不得实施以不正当的低价销售商品、串通投标、发布虚假广告、进行商业贿赂等不正当竞争行为。 在对外贸易经营活动中实施不正当竞争行为的，依照有关反不正当竞争的法律、行政法规的规定处理。
有前款违法行为，并危害对外贸易秩序的，国务院对外贸易主管部门可以采取禁止该经营者有关货物、技术进出口等措施消除危害。
Article: 第三十三条
Section: 第六章 对外贸易秩序
Name: 中华人民共和国对外贸易法
Score: 1.463229

Content: 第四条 社区矫正对象应当依法接受社区矫正，服从监督管理。 社区矫正工作应当依法进行，尊重和保障人权。社区矫正对象依法享有的人身权利、财产权利和其他权利不受侵犯，在就业、就学和享受社会保障等方面不受歧视。
Article: 第四条
Section: 第一章  总则
Name: 中华人民共和国社区矫正法
Score: 1.4607658

Content: 第二十条 为了使国家、公共利益、本人或者他人的人身、财产和其他权利免受正在进行的不法侵害，而采取的制止不法侵害的行为，对不法侵害人造成损害的，属于正当防卫，不负刑事责任。
正当防卫明显超过必要